# Optimización con Vectorización, Profiling y JIT/AOT en Python
Este notebook muestra técnicas de optimización: vectorización con NumPy/PyTorch, profiling, benchmarking, JIT con Numba y AOT (caching y Cython).

## Conceptos Clave de Vectorización y Eficiencia Computacional

La vectorización no solo simplifica el código, sino que proporciona **mejoras sustanciales en rendimiento** al aprovechar:
- **Instrucciones SIMD**: Modernas CPUs incluyen extensiones (SSE, AVX, NEON) que ejecutan la misma operación sobre múltiples datos en un solo ciclo, procesando en paralelo vectores de 4, 8 o más valores.
- **Operaciones en C optimizado**: Las funciones universales de NumPy (ufuncs) están escritas en C, evitando el overhead del intérprete Python en cada iteración.
- **Localidad de memoria**: Al trabajar con arrays contiguos, se maximizan los “cache hits” de L1/L2/L3, reduciendo dramáticamente los accesos a memoria RAM.
- **BLAS/MKL/OpenBLAS multihilo**: Para álgebra lineal pesada (`np.dot`, `np.linalg.inv`), estas bibliotecas distribuyen el cómputo entre varios núcleos, equilibrando carga y minimizando tiempos de espera.
- **Broadcasting**: Permite operar sin crear copias innecesarias de arrays, reusando la misma memoria y evitando loops Python implícitos.
- **Menos GIL**: Al delegar todo el trabajo en C (o en GPU para PyTorch/TensorFlow), se liberan cuellos de botella asociados al Global Interpreter Lock.
- **GPU y Tensor Cores**: En Deep Learning, frameworks vectorizados extienden estos principios al hardware de GPU, usando miles de núcleos paralelos y Tensor Cores para acelerar multiplicaciones de matrices a precisión reducida (FP16/TF32).

> **Reflexión**: Al vectorizar, transformas tu código de una serie de bucles Python en una composición de operaciones de alto nivel ejecutadas directamente en hardware especializado, logrando **ganancias de 10× a 100×** en muchas tareas numéricas.

## Detalle del Pipeline de Optimización

1. **Identificación de cuellos de botella**: Usa `%timeit`, `cProfile` o `line_profiler` para detectar loops costosos.
2. **Reemplazo por ufuncs**: Sustituye cada iteración por llamadas a `np.add`, `np.multiply`, `np.dot`, etc.
3. **Validación de shapes**: Al usar broadcasting, verifica que las dimensiones sean compatibles para evitar bugs silenciosos.
4. **Ajuste de dtype**: Selecciona tipos de datos más ligeros (`float32` vs `float64`) para reducir carga de memoria y acelerar cálculos.
5. **Test de rendimiento**: Mide antes y después de cada cambio para asegurar que la vectorización aporta mejoras reales.

## Implementación: Generación de Datos, Modelo y Vectorización
A continuación el código completo con docstrings y uso de inferencia vectorizada.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import time

def generar_datos_sinteticos(num_muestras=5000):
    """
    Genera datos sintéticos para clasificación binaria.
    """
    X = np.random.randn(num_muestras, 10)
    y = (np.sum(X, axis=1) > 0).astype(int)
    return X, y

class RedNeuronalSimple(nn.Module):
    """Red neuronal de dos capas."""
    def __init__(self, dim_entrada=10, dim_oculta=32, dim_salida=2):
        super().__init__()
        self.modelo = nn.Sequential(
            nn.Linear(dim_entrada, dim_oculta), nn.ReLU(), nn.Linear(dim_oculta, dim_salida)
        )
    def forward(self, x): return self.modelo(x)

def entrenar_modelo(modelo, X_train, y_train, epochs=5, batch_size=128):
    """Entrena con Adam y CrossEntropyLoss."""
    optimizer = optim.Adam(modelo.parameters(), lr=1e-3)
    criterio = nn.CrossEntropyLoss()
    modelo.train()
    tensor_X = torch.tensor(X_train, dtype=torch.float32)
    tensor_y = torch.tensor(y_train, dtype=torch.long)
    dataset = torch.utils.data.TensorDataset(tensor_X, tensor_y)
    loader  = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    for epoch in range(epochs):
        pérdida=0
        for bx,by in loader:
            optimizer.zero_grad(); out=modelo(bx); loss=criterio(out, by)
            loss.backward(); optimizer.step(); pérdida+=loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Pérdida Promedio: {pérdida/len(loader):.4f}")

def inferencia_vectorizada(modelo, X_data, batch_size=None, device='cpu'):
    """Inferencia en lotes grandes sin hilos manuales."""
    modelo.to(device).eval()
    with torch.no_grad():
        tX = torch.tensor(X_data, dtype=torch.float32, device=device)
        if batch_size:
            preds=[]
            for i in range(0, len(tX), batch_size): preds.append(
                modelo(tX[i:i+batch_size]).argmax(dim=1).cpu()
            )
            return torch.cat(preds).numpy()
        out = modelo(tX)
        return out.argmax(dim=1).cpu().numpy()

def main():
    X,y=generar_datos_sinteticos(5000)
    s=int(0.8*len(X)); X_tr,X_te=X[:s],X[s:]; y_tr,y_te=y[:s],y[s:]
    mod=RedNeuronalSimple()
    print("Entrenando..."); t0=time.time(); entrenar_modelo(mod,X_tr,y_tr)
    print(f"Entrenamiento en {time.time()-t0:.2f}s")
    print("Inferencia..."); t1=time.time(); p=inferencia_vectorizada(mod,X_te)
    print(f"Inferencia en {time.time()-t1:.2f}s")
    print(f"Precisión: {(p==y_te).mean()*100:.2f}%")

if __name__=='__main__': main()

## Benchmarking con `time.perf_counter`
Medimos tiempos de entrenamiento e inferencia en ejecuciones rápidas de prueba.

In [ ]:
import time
# Setup rápido
X,y = generar_datos_sinteticos(5000)
s=int(0.8*len(X)); Xt, Xv = X[:s], X[s:]; yt, yv = y[:s], y[s:]
mod = RedNeuronalSimple()
# Entrenamiento 1 epoch
t0=time.perf_counter(); entrenar_modelo(mod, Xt, yt, epochs=1); print('Train:',time.perf_counter()-t0)
# Inferencia completa
t1=time.perf_counter(); inferencia_vectorizada(mod, Xv); print('Infer:',time.perf_counter()-t1)

## Perfilado con `cProfile`
Detectamos funciones con mayor tiempo de ejecución.

In [ ]:
import cProfile, pstats
prof = cProfile.Profile()
prof.enable()
main()
prof.disable()
pstats.Stats(prof).sort_stats('cumtime').print_stats(10)

## Análisis línea a línea con `line_profiler`
1. Instalar: `pip install line_profiler`
2. Decorar funciones críticas con `@profile`
3. Ejecutar: `kernprof -l -v vectorizacion_completo.ipynb`

## Just-In-Time (JIT) con Numba
Con Numba podemos compilar funciones de Python a código máquina en tiempo de ejecución, logrando aceleraciones muy significativas en bucles Python:
- `@njit`: compila en modo nopython, sin dependencia del intérprete.
- Primera llamada incurre en overhead de compilación; posteriores, el código compilado se ejecuta rápido.


In [ ]:
from numba import njit
import numpy as np
import time

@njit
def calc_distances_numba(points, center):
    n = points.shape[0]
    out = np.empty(n, dtype=np.float64)
    for i in range(n):
        diff0 = points[i, 0] - center[0]
        diff1 = points[i, 1] - center[1]
        diff2 = points[i, 2] - center[2]
        out[i] = (diff0*diff0 + diff1*diff1 + diff2*diff2) ** 0.5
    return out

# Dataset de prueba
points = np.random.rand(100000, 3)
center = np.array([0.5, 0.5, 0.5])

# Calentamiento inicial (compilación)
calc_distances_numba(points[:1000], center)

# Medición de rendimiento
t0 = time.perf_counter()
dists_numba = calc_distances_numba(points, center)
t1 = time.perf_counter()
print(f"Numba JIT tiempo: {t1 - t0:.4f} s")

# Vectorizado NumPy para comparar
t2 = time.perf_counter()
dists_np = np.sqrt(((points - center) ** 2).sum(axis=1))
t3 = time.perf_counter()
print(f"NumPy vectorizado tiempo: {t3 - t2:.4f} s")


## Ahead-of-Time (AOT) y Compilación Persistente
- **Cache de Numba**: usando `@njit(cache=True)` almacenamos el binario compilado en disco,
  reduciendo el overhead en ejecuciones futuras.
- **Cython**: herramienta clásica para compilación AOT, transformando código Python tipado en C.
  Requiere compilación previa, pero ofrece máximo rendimiento en secciones críticas.


In [ ]:
from numba import njit
import numpy as np
import time

@njit(cache=True)
def calc_distances_numba_cached(points, center):
    n = points.shape[0]
    out = np.empty(n, dtype=np.float64)
    for i in range(n):
        diff0 = points[i, 0] - center[0]
        diff1 = points[i, 1] - center[1]
        diff2 = points[i, 2] - center[2]
        out[i] = (diff0*diff0 + diff1*diff1 + diff2*diff2) ** 0.5
    return out

# Ejecución para demostrar cache
points = np.random.rand(100000, 3)
center = np.array([0.5, 0.5, 0.5])
print("Compilación y primera ejecución (incluye compilación)...")
calc_distances_numba_cached(points[:1000], center)
print("Ejecución posterior recupera compilado sin overhead significativo")
t0 = time.perf_counter()
calc_distances_numba_cached(points, center)
print(f"Numba JIT con cache tiempo: {time.perf_counter() - t0:.4f} s")
